In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def get_rasp_header_from_user():
    """Solicita ao usuário as informações para a linha de cabeçalho RASP."""
    print("Por favor, insira as informações para o cabeçalho do arquivo RASP:")

    # 1. Nome comum do motor
    motor_name = input("Nome comum do motor: ")

    # 2. Diâmetro do invólucro (mm)
    while True:
        try:
            diameter_str = input("Diâmetro do invólucro em mm: ")
            diameter = float(diameter_str.replace(',', '.')) # Aceita vírgula como decimal
            break
        except ValueError:
            print("Entrada inválida. Por favor, insira um número para o diâmetro.")

    # 3. Comprimento do invólucro (mm)
    while True:
        try:
            length_str = input("Comprimento do invólucro em mm: ")
            length = float(length_str.replace(',', '.'))
            break
        except ValueError:
            print("Entrada inválida. Por favor, insira um número para o comprimento.")

    # 4. Atrasos disponíveis
    delays = input("Atrasos disponíveis (ex: 6-10-14, ou 0, ou P para 'plugged'): ")

    # 5. Peso do propelente (Kg)
    while True:
        try:
            prop_weight_str = input("Peso do propelente em Kg: ")
            prop_weight = float(prop_weight_str.replace(',', '.'))
            break
        except ValueError:
            print("Entrada inválida. Por favor, insira um número para o peso do propelente.")

    # 6. Peso total do motor (Kg)
    while True:
        try:
            total_weight_str = input("Peso total do motor carregado em Kg: ")
            total_weight = float(total_weight_str.replace(',', '.'))
            break
        except ValueError:
            print("Entrada inválida. Por favor, insira um número para o peso total.")

    # 7. Abreviação do fabricante
    manufacturer = input("Abreviação do fabricante: ")

    # Formata a linha de cabeçalho
    # Ex: K550 54 300 6-10-14 0.250 0.500 AT
    header_line = f"{motor_name} {diameter} {length} {delays} {prop_weight:.4f} {total_weight:.4f} {manufacturer}"
    return header_line

In [14]:
def format_dataframe_to_rasp_data(dataframe):
    """Formata os dados de um DataFrame para o formato de dados RASP."""
    rasp_data_lines = []
    # A especificação RASP diz que um ponto (0,0) é implícito e não deve ser listado.
    # Se o nosso DataFrame tem um ponto (0,0) como primeira linha (adicionado para plotagem),
    # podemos querer pulá-lo aqui, a menos que o primeiro ponto de dados real seja em t=0.
    
    start_index = 0
    if not dataframe.empty and dataframe['tempo'].iloc[0] == 0.0 and dataframe['empuxo'].iloc[0] == 0.0:
        # Se o primeiro ponto é (0,0) e há mais pontos, ou se é o único ponto (improvável para dados reais)
        # e o próximo ponto também é em t=0 mas com empuxo > 0 (como no seu CSV original),
        # este (0,0) é o implícito.
        # No entanto, se o *primeiro dado medido* no arquivo original já era t=0, empuxo>0,
        # então esse deve ser o primeiro ponto no arquivo RASP.

        # Se o seu df tem [ (0,0), (0, E1), (T2,E2)...] onde (0,0) foi adicionado
        # e (0,E1) veio do seu CSV com Tempo=0 e Empuxo=E1.
        # O RASP ideal teria (0,E1) como primeira linha.
        # Vamos verificar se o segundo ponto também é tempo 0, o que indica que (0,0) foi adicionado
        # antes de um ponto de dados real em t=0.
        if len(dataframe) > 1 and dataframe['tempo'].iloc[1] == 0.0:
            start_index = 1 # Pula o (0,0) artificial e começa do (0, E_real)
        # Se o primeiro ponto é (0,0) e o segundo é (T_real >0, E_real), então o (0,0) é implícito
        # e não deve ser incluído.
        elif len(dataframe) > 1 and dataframe['tempo'].iloc[1] > 0.0:
             start_index = 1 # Pula o (0,0) artificial

    for i in range(start_index, len(dataframe)):
        row = dataframe.iloc[i]
        # Formata com espaços no início para legibilidade, como é comum
        # Ajuste os {:.3f} conforme a precisão desejada.
        rasp_data_lines.append(f"{row['tempo']:.3f} {row['empuxo']:.4f}")
        
    # Garante que o último ponto tenha empuxo zero
    if not dataframe.empty:
        last_row = dataframe.iloc[-1]
        if last_row['empuxo'] != 0.0:
            print("\nAviso: O último ponto de dados no DataFrame não tem empuxo zero.")
            print("Um arquivo RASP válido requer que o último ponto de empuxo seja zero.")
            # Você pode optar por adicionar automaticamente um ponto (tempo_ultimo, 0)
            # ou modificar o último ponto, mas isso alteraria os dados originais.
            # Por agora, apenas avisamos. A função `rasp_to_dataframe`
            # já garantia isso para o DataFrame.

    return rasp_data_lines

In [15]:
# --- Início da Execução Principal ---

# 1. Crie ou carregue seu DataFrame aqui.
# Vou usar um exemplo baseado no seu '50mm_dados.csv'
# Supondo que o df já foi processado conforme os passos anteriores:
# - Carregado de '50mm_dados.csv'
# - Colunas 'Data', 'Hora' removidas
# - Renomeado para 'tempo' e 'empuxo'
# - Ponto (0.0, 0.0) adicionado no início se não existia
# - Ordenado por 'tempo'

# Exemplo de criação do DataFrame (simulando o seu caso):
data_simulada = {
    'tempo': [0.0, 0.0, 0.095, 0.19, 0.284, 2.087, 2.182, 2.182], # Adicionei um t=0, e=0 e um t=2.182, e=0
    'empuxo': [0.0, 0.2453, 0.5886, 0.8044, 0.9614, 0.8633, 0.412, 0.0] # Último empuxo é 0
}
df_motor = pd.DataFrame(data_simulada)
df_motor = df_motor.sort_values(by='tempo').reset_index(drop=True)
# Removendo duplicatas de tempo, mantendo o último (para casos como (0,0) e (0, E1))
# Se o (0,0) foi adicionado e o primeiro dado era (0, E_real), queremos ambos para mostrar a subida.
# Se o (0,0) foi adicionado e o primeiro dado era (T_real > 0, E_real), então o (0,0) é implícito e não impresso.
# A lógica em format_dataframe_to_rasp_data tenta lidar com isso.

# Se o seu primeiro ponto no CSV é (0.0, EmpuxoReal), e você adicionou (0.0, 0.0) antes:
# df_motor começaria com:
#   tempo  empuxo
#0    0.0  0.0000
#1    0.0  0.2453
# ...
# O format_dataframe_to_rasp_data pularia a linha 0 e começaria a imprimir "0.000 0.2453"

# Caso real usando seu arquivo '50mm_dados.csv' (coloque o arquivo no mesmo diretório)
try:
    df_real = pd.read_csv('40mm_dados.csv', sep=';')
    df_real = df_real.drop(columns=['Data', 'Hora'])
    df_real = df_real.rename(columns={'Tempo': 'tempo', 'Empuxo': 'empuxo'})
    df_real['tempo'] = pd.to_numeric(df_real['tempo'], errors='coerce')
    df_real['empuxo'] = pd.to_numeric(df_real['empuxo'], errors='coerce')
    df_real.dropna(subset=['tempo', 'empuxo'], inplace=True)
    
    if not (df_real.empty or (df_real['tempo'].iloc[0] == 0 and df_real['empuxo'].iloc[0] == 0)):
        ponto_inicial = pd.DataFrame({'tempo': [0.0], 'empuxo': [0.0]})
        df_real = pd.concat([ponto_inicial, df_real]).reset_index(drop=True)
    
    df_real = df_real.sort_values(by='tempo').reset_index(drop=True)

    # Garante que o último ponto tenha empuxo zero, adicionando se necessário
    if not df_real.empty and df_real['empuxo'].iloc[-1] != 0.0:
        # Adiciona um novo ponto com o mesmo tempo do último, mas com empuxo zero
        # ou incrementa o tempo do último ponto ligeiramente para o ponto zero.
        # Por simplicidade, vamos assumir que o último ponto do CSV já representa o fim da queima visível
        # e o próximo é zero. Se seu CSV é `... Tn, En` e En !=0, então falta `Tn_fim, 0`.
        # No seu `50mm_dados.csv`, o último ponto não é zero. Vamos adicionar um.
        # O último tempo é 2.182 com empuxo 0.412.
        # O ideal é que o processo que gerou o CSV já tivesse o ponto de empuxo zero final.
        # Adicionando um ponto para garantir conformidade RASP:
        df_real = pd.concat([df_real, pd.DataFrame({'tempo': [df_real['tempo'].iloc[-1] + 0.001], 'empuxo': [0.0]})], ignore_index=True)
        df_real = df_real.sort_values(by='tempo').reset_index(drop=True)
        
    df_para_rasp = df_real # Usar o DataFrame carregado do arquivo real
    print("\nDataFrame carregado e preparado (primeiras e últimas linhas):")
    print(pd.concat([df_para_rasp.head(3), df_para_rasp.tail(3)]))

except FileNotFoundError:
    print("\nArquivo '50mm_dados.csv' não encontrado. Usando dados simulados para demonstração.")
    df_para_rasp = df_motor # Usar o DataFrame simulado se o arquivo real não for encontrado
    print("\nDataFrame simulado (primeiras e últimas linhas):")
    print(pd.concat([df_para_rasp.head(3), df_para_rasp.tail(3)]))


# 2. Obter a linha de cabeçalho do usuário
rasp_header = get_rasp_header_from_user()

# 3. Formatar os dados do DataFrame para o formato RASP
rasp_data = format_dataframe_to_rasp_data(df_para_rasp)

# 4. Montar o conteúdo completo do arquivo RASP
print("\n--- Conteúdo do Arquivo RASP Gerado ---")
# Adicionar alguns comentários iniciais (opcional)
print("; Arquivo .eng gerado por script Python")
print(f"; Motor: {rasp_header.split()[0]}") # Pega o nome do motor do cabeçalho
print(";") # Linha de comentário em branco

print(rasp_header) # Imprime a linha de cabeçalho

for data_line in rasp_data:
    print(data_line)

print(";") # Comentário final para separar entradas em um arquivo multi-motor
print("--- Fim do Conteúdo ---")

# Opcional: Salvar em um arquivo
rasp_file_name = f"{rasp_header.split()[0]}_{rasp_header.split()[-1]}.eng"
with open(rasp_file_name, "w") as f:
    f.write("; Arquivo .eng gerado por script Python\n")
    f.write(f"; Motor: {rasp_header.split()[0]}\n")
    f.write(";\n")
    f.write(rasp_header + "\n")
    for data_line in rasp_data:
        f.write(data_line + "\n")
f.close()
print(f"\nArquivo RASP salvo como: {rasp_file_name}")


DataFrame carregado e preparado (primeiras e últimas linhas):
    tempo  empuxo
0   0.000  0.0000
1   0.000  0.1373
2   0.096  0.4218
30  2.900  0.3335
31  2.993  0.2747
32  2.994  0.0000
Por favor, insira as informações para o cabeçalho do arquivo RASP:

--- Conteúdo do Arquivo RASP Gerado ---
; Arquivo .eng gerado por script Python
; Motor: Teste_40mm
;
Teste_40mm 17.0 40.0 P 0.0134 0.0334 Custom
0.000 0.1373
0.096 0.4218
0.190 0.5003
0.286 0.5592
0.380 0.6377
0.475 0.7456
0.570 0.8535
0.665 0.9908
0.760 1.1282
0.871 1.2949
1.048 1.6285
1.139 1.8148
1.235 2.0110
1.329 2.0405
1.425 1.9914
1.519 1.9522
1.615 1.8737
1.709 1.7462
1.805 1.5696
1.899 1.3538
1.994 1.1478
2.088 0.9418
2.184 0.8142
2.278 0.7161
2.374 0.6180
2.468 0.5592
2.579 0.5101
2.717 0.4513
2.808 0.3826
2.900 0.3335
2.993 0.2747
2.994 0.0000
;
--- Fim do Conteúdo ---

Arquivo RASP salvo como: Teste_40mm_Custom.eng
